Here, we'll try to fit our very first model on TennisExplorer data.

OTHER TODO: 
* Figure out walkover / retirement handling!
    * Try 2x2 training and evaluation
* Try making formulaic match predictions for set model, e.g.:
* Weight tiebreak sets differently
* Try weighting by games


$\begin{align}
    \mathbb{P}_2 &= \hat{p}^2(1 + 2(1 - \hat{p})) \\
    \mathbb{P}_3 &= \hat{p}^3(1 + 3(1 - \hat{p}) + {4 \choose 2}(1 - \hat{p})^2)
\end{align} \\
\text{where } \hat{p} \text{ is the set prediction}
$

Note that the bottom TODO will not affect accuracy results, but may help with the AUC metrics and later calibration and betting performance.

In [1]:
from tennis_new.fetch.tennis_explorer.combiner import read_joined

jd = read_joined()

/Users/siddhantjagadish/Documents/DataProjects/tennis_new/venv_377/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3214: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):


In [2]:
from tennis_new.model.utils.filters import TrainingFilter

rel = TrainingFilter().filter_data(jd)

In [3]:
from tennis_new.model.config.elo.base import ELOBaseModel 

gs_elo = ELOBaseModel()
gs_elo.run(jd)

In [4]:
gs_elo.validation_evaluation

{'DummyFilter_prediction_AUCMetric': 0.8088814270063787,
 'DummyFilter_prediction_AccuracyMetric': 0.7275996830794705,
 'DummyFilter_prediction_LogLikelihoodMetric': -0.533704037104847,
 'HasOddsFilter_prediction_AUCMetric': 0.7849735388169738,
 'HasOddsFilter_prediction_AccuracyMetric': 0.7074658387051017,
 'HasOddsFilter_prediction_LogLikelihoodMetric': -0.556931976858201,
 'DummyFilter_odds_implied_probability_AUCMetric': None,
 'DummyFilter_odds_implied_probability_AccuracyMetric': None,
 'DummyFilter_odds_implied_probability_LogLikelihoodMetric': None,
 'HasOddsFilter_odds_implied_probability_AUCMetric': 0.7955634245248138,
 'HasOddsFilter_odds_implied_probability_AccuracyMetric': 0.7200428690759507,
 'HasOddsFilter_odds_implied_probability_LogLikelihoodMetric': -0.5484675296646053}

In [5]:
from tennis_new.model.config.elo.global_set_elo import SetELO

set_elo = SetELO()
set_elo.run(jd)
set_elo.validation_evaluation

{'DummyFilter_prediction_AUCMetric': 0.8187031847302881,
 'DummyFilter_prediction_AccuracyMetric': 0.7358520800135314,
 'DummyFilter_prediction_LogLikelihoodMetric': -0.5226366377611569,
 'HasOddsFilter_prediction_AUCMetric': 0.786068092124641,
 'HasOddsFilter_prediction_AccuracyMetric': 0.7080332235338619,
 'HasOddsFilter_prediction_LogLikelihoodMetric': -0.5576990341361893,
 'DummyFilter_odds_implied_probability_AUCMetric': None,
 'DummyFilter_odds_implied_probability_AccuracyMetric': None,
 'DummyFilter_odds_implied_probability_LogLikelihoodMetric': None,
 'HasOddsFilter_odds_implied_probability_AUCMetric': 0.7955634245248138,
 'HasOddsFilter_odds_implied_probability_AccuracyMetric': 0.7200428690759507,
 'HasOddsFilter_odds_implied_probability_LogLikelihoodMetric': -0.5484675296646053}

#### P-Test for Accuracy Improvement
Let's perform a P-Test to Ensure that our accuracy has improved!

In [6]:
test_set = gs_elo.test_filter.filter_data(gs_elo.all_jd)
set_test_set = set_elo.test_filter.filter_data(set_elo.all_jd)

In [7]:
from statsmodels.stats.proportion import proportions_ztest

proportions_ztest(
    [(test_set['prediction'] > 0.5).sum(), (set_test_set['prediction'] > 0.5).sum()],
    [len(test_set), len(set_test_set)],
)[-1]

1.2167284746968122e-09

Hooray, we have a statistically significant accuracy improvement!